In [1]:
pip install minsearch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [15]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [26]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [25]:
SELECT * WHERE course = 'data-engineering-zoomcamp'

SyntaxError: invalid syntax (2183464379.py, line 1)

In [27]:
q = 'the course has already started, can I still enroll?'

In [28]:
index.fit(documents)

In [46]:
boost = {'question': 3.0, 'section': 0.5} 

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [32]:
from groq import Groq

In [33]:
client = Groq()

In [34]:
response = client.chat.completions.create(
    model = 'meta-llama/llama-4-scout-17b-16e-instruct',
    messages = [{"role": "user", "content": q}]
)

In [41]:
response.choices[0].message.content

"The answer depends on the specific course and institution you're interested in. Here are some general possibilities:\n\n1. **Late enrollment**: Some courses or programs may allow late enrollment, even after the course has started. This is more common in online courses or those with a flexible schedule. However, you may need to catch up on missed material or assignments.\n2. **Waitlisting**: If the course is full, you might be able to join a waitlist. If a spot becomes available (e.g., someone drops out), you may be able to enroll.\n3. **Permission from the instructor**: In some cases, you can ask the instructor if they can still accept you into the course. This is more likely to happen if the course is not too far along, and the instructor believes you can catch up.\n4. **No late enrollment**: Unfortunately, some courses or programs may have a strict enrollment deadline, and late enrollment is not allowed. This is often the case for courses with a specific sequence or where materials 

In [59]:
prompt_template = """    
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [47]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [51]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [64]:
prompt = prompt_template.format(question=q, context=context).strip()

In [65]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.



In [68]:
response = client.chat.completions.create(
    model = 'meta-llama/llama-4-scout-17b-16e-instruct',
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. \n\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [84]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5} 

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [92]:
def build_prompt(query, context):
    prompt_template = """    
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt
    

In [93]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'meta-llama/llama-4-scout-17b-16e-instruct',
        messages = [{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [99]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [100]:
query = 'How do I run kafka?'

rag(query)

'NONE \n\n(The CONTEXT does not contain information on how to run Kafka. It only provides general information about the course, prerequisites, and support.)'